In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

from google.colab import drive

drive.mount('/content/drive')

import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
!apt-get install abcmidi timidity > /dev/null 2>&1
import requests
import librosa


os.makedirs('/content/drive/My Drive/drakemp3', exist_ok=True)

audio_paths = [
    '/content/drive/My Drive/drakemp3/Currents.mp3',
    '/content/drive/My Drive/drakemp3/TextsGoGreen.mp3',
    '/content/drive/My Drive/drakemp3/FallingBack.mp3'
]


spectrogram_folder = '/content/drake_spectrograms'
os.makedirs(spectrogram_folder, exist_ok=True)

for audio_path in audio_paths:
    y, sr = librosa.load(audio_path, sr=44100)
    spectrogram = np.abs(librosa.stft(y))
    np.save(os.path.join(spectrogram_folder, f'{os.path.splitext(os.path.basename(audio_path))[0]}.npy'), spectrogram)


spectrogram_folder = '/content/drake_spectrograms'
os.makedirs(spectrogram_folder, exist_ok=True)

for audio_file in os.listdir('/content/drake_songs'):
    audio_path = os.path.join('/content/drake_songs', audio_file)
    y, sr = librosa.load(audio_path, sr=44100)
    spectrogram = np.abs(librosa.stft(y))
    np.save(os.path.join(spectrogram_folder, f'{os.path.splitext(audio_file)[0]}.npy'), spectrogram)



gpus = tf.config.list_physical_devices('GPU')

if len(gpus) > 0:
    print("GPU is available.")

else:
    print("No GPU detected. Running on CPU.")



Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Mounted at /content/drive


FileNotFoundError: ignored

In [ ]:
# Load segmented spectrograms
spectrogram_folder = '/content/drake_spectrograms'
segmented_folder = '/content/drake_segmented'

os.makedirs(segmented_folder, exist_ok=True)

# Define segment length and hop length
segment_length = 100  # Define the length of each segment
hop_length = 50  # Define hop length for segmentation

for spectrogram_file in os.listdir(spectrogram_folder):
    spectrogram_path = os.path.join(spectrogram_folder, spectrogram_file)
    spectrogram = np.load(spectrogram_path)

    # Segment the spectrogram
    num_segments = (spectrogram.shape[1] - segment_length) // hop_length

    for i in range(num_segments):
        segment = spectrogram[:, i * hop_length : i * hop_length + segment_length]
        np.save(os.path.join(segmented_folder, f'{os.path.splitext(spectrogram_file)[0]}_{i}.npy'), segment)


In [ ]:
# Define and train RNN (LSTM for example)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load segmented spectrograms
segmented_folder = '/content/drake_segmented'

# Collect segmented spectrograms
segments = []
for segmented_file in os.listdir(segmented_folder):
    segmented_path = os.path.join(segmented_folder, segmented_file)
    segment = np.load(segmented_path)
    segments.append(segment)

# Convert segments to numpy array
segments = np.array(segments)

# Calculate num_features based on the shape of segments
num_features = segments.shape[1] * segments.shape[2]

# Define RNN model
model = Sequential()
model.add(LSTM(units=128, input_shape=segments.shape[1:], return_sequences=True))
model.add(Dense(units=segments.shape[2]))  # Output layer matching segment_length

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(segments, segments, epochs=10, batch_size=32)  # Adjust epochs and batch_size as needed



In [ ]:
!pip install nltk

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from google.colab import drive
import pandas as pd
import re
from nltk.translate.bleu_score import sentence_bleu

drive.mount('/content/drive')

# Function to read text from a CSV file
def read_text_from_csv(file_path):
    data = pd.read_csv(file_path)
    lyrics = data['lyrics'].str.lower().str.cat(sep=' ')
    return lyrics

# Path to the CSV file containing additional Drake lyrics
lyrics_csv_path = '/content/drive/My Drive/drakemp3/drake_data.csv'
drake_lyrics_additional = read_text_from_csv(lyrics_csv_path)

# Path to the existing text file of Drake's lyrics
lyrics_file_path = '/content/drive/My Drive/drakemp3/drake-lyrics.txt'
drake_lyrics = read_text_from_file(lyrics_file_path)

# Combining the additional lyrics with the existing ones
drake_lyrics += drake_lyrics_additional


# Check GPU availability
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

# Function to read text from a single .txt file
def read_text_from_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read().lower()
    return text

lyrics_file_path = '/content/drive/My Drive/drakemp3/drake-lyrics.txt'
drake_lyrics = read_text_from_file(lyrics_file_path)

# Create a character-level mapping
chars = sorted(list(set(drake_lyrics)))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
idx_to_char = {idx: ch for idx, ch in enumerate(chars)}

max_len = 250  # Increased sequence length
step = 5  # Increased step size for overlapping sequences
sequences = []
next_chars = []

for i in range(0, len(drake_lyrics) - max_len, step):
    sequences.append(drake_lyrics[i:i + max_len])
    next_chars.append(drake_lyrics[i + max_len])

# Vectorize sequences
x = np.zeros((len(sequences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char_to_idx[char]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1

# TensorFlow configuration to utilize GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Build an RNN model with increased complexity
model = Sequential()
model.add(LSTM(256, input_shape=(max_len, len(chars)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model for more epochs
model.fit(x, y, batch_size=256, epochs=75)  # Increased epochs

# Generate text using the trained model with adjusted temperature
seed_text = "I'm feeling"  # Provide a seed text to start generating
generated_text = seed_text.lower()

temperature = 0.2  # Adjust temperature for diversity in text generation

for _ in range(1000):  # Generate longer lyrics
    sampled = np.zeros((1, max_len, len(chars)))
    for t, char in enumerate(generated_text[-max_len:]):
        sampled[0, t, char_to_idx[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0]
    next_index = np.random.choice(len(chars), p=preds)
    next_char = idx_to_char[next_index]

    generated_text += next_char

# Post-processing for removing unwanted characters and repeated sequences
generated_text = re.sub(r'[^a-zA-Z\s]', '', generated_text)
words = generated_text.split()
filtered_words = [words[i] for i in range(len(words)) if i == 0 or words[i] != words[i - 1]]
generated_text = ' '.join(filtered_words)





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Num GPUs Available: 1


<ipython-input-11-6bc3615e4549>:57: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sequences), max_len, len(chars)), dtype=np.bool)
<ipython-input-11-6bc3615e4549>:58: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sequences), len(chars)), dtype=np.bool)


Epoch 1/75


Exception ignored in: <function _xla_gc_callback at 0x7f73f52c7640>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


237/237 [==============================] - 77s 93ms/step - loss: 3.0277
Epoch 2/75
237/237 [==============================] - 22s 91ms/step - loss: 2.6361
Epoch 3/75
237/237 [==============================] - 21s 89ms/step - loss: 2.3595
Epoch 4/75
237/237 [==============================] - 21s 90ms/step - loss: 2.2564
Epoch 5/75
237/237 [==============================] - 22s 91ms/step - loss: 2.1793
Epoch 6/75
237/237 [==============================] - 22s 91ms/step - loss: 2.1155
Epoch 7/75
237/237 [==============================] - 21s 90ms/step - loss: 2.0638
Epoch 8/75
237/237 [==============================] - 21s 91ms/step - loss: 2.0200
Epoch 9/75
237/237 [==============================] - 22s 91ms/step - loss: 1.9792
Epoch 10/75
237/237 [==============================] - 21s 91ms/step - loss: 1.9463
Epoch 11/75
237/237 [==============================] - 22s 91ms/step - loss: 1.9090
Epoch 12/75
237/237 [==============================] - 21s 91ms/step - loss: 1.8765
Epoch 13/75


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
seed_text = "I'm feeling good"  # Provide a seed text to start generating
generated_text = seed_text.lower()

temperature = .1  # Adjust temperature for diversity in text generation

for _ in range(1000):  # Generate longer lyrics
    sampled = np.zeros((1, max_len, len(chars)))
    for t, char in enumerate(generated_text[-max_len:]):
        sampled[0, t, char_to_idx[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0]
    next_index = np.random.choice(len(chars), p=preds)
    next_char = idx_to_char[next_index]

    generated_text += next_char

print(generated_text)

i'm feeling goodt

y.dhamobfdowam
faaetpee-aif
baota.
ttbida
endaaat

as
deebytp
.tde
wgdstta
 d

aaptbbwoahawwaaab
.tayw
ad.eeedw
atb
.aaaauatshhna'nd micain
yea.
but i know we know i don't know we're a simper i meat with it me always so it's on, where it wat am... (ohah, oover toden toog.

i swou?
i just knomadin how you just been here, i'm gettin' plissen
we always still
i'm clot ying this shit, and what yea's won you about whenent their out of men
if chish goode rigghas back fou up
trying to the conight
relatee dargit's somethin money would get really winnat so hopies (someded you would tige, i'm too good, getting be just can't be done like you chough but i want to trebotess to be lougnt than you ain't you
lights better that's been extine it bigga
just want we can cur some stripper
i get it and cause wat't don't have for you
this she was her lasters you told it
girl, and i've bad going nigga
do to be a nigga, i ain't no lever when i king throught up to boprotite
why i call as a tir